# Twin Cities Neighborhood Analysis

What I want to do.
1. Calculate the distance of all of them in relation to the airport and Oronoco.
2. Verify the miles calculation by spot checking a few with Google Maps
3. Find a way to overlay information (e.g. distance) on the shapes or popup.
3. Pull in other metadata (e.g. real estate, crime, water, etc...)

## Projection Summary
The city shapes are provided by the US Census Bureau in a geographic CRS NAD83 (authority code: [EPSG:4269](https://epsg.io/4269)). 
A _geographic_ projection is one in which that the coordinates are in latitude and longitude.
The US Census Bureau uses NAD83 for most of their stuff. This is an un-projected datum.
See this [article](https://source.opennews.org/articles/choosing-right-map-projection/) for more details.

A _projected_ CRS is in meters, feet, kilometers etc. (Note: You can see the 
active projection with the _crs_ property on a _GeoDataFrame_ object.)

The provided CRS -- EPSG:4269 -- isn't good for doing distance calculations. We need to 
use a equi-distant projection. To enable accurate calculations we use 
EPSG:4087 with is the  WGS 84 / World Equidistant Cylindrical CRS. 
This projection's UOM is in meters.

The popular online mapping systems (e.g. Google, Bing, Apple) all use [EPSG:3857](https://epsg.io/3857).
The [Wikipedia page](https://en.wikipedia.org/wiki/Web_Mercator_projection) has 
details on the pros and cons of this projection.

**To Clarify**
1. The spatial shapes are loaded in EPSG:4269.
2. The geometry is re-projected to EPSG:4087 to enable calculations.
3. Once the data is ready to be rendered on a map it is projected again to EPSG:3857.

In [177]:
from IPython.display import display
import geopandas as gdf
import pandas as pd
import folium
import matplotlib as mp

# Load the data files into DataFrames
mn_counties_shapes = gdf.read_file('./data/cb_2018_27_cousub_500k.zip')

"""
TODO
Merge real estate data.
- [X] Associate real estate ZIP codes with GEOIDs
- [X] Import real estate data as a Pandas dataframe.
- [ ] Find a way to merge on NAME + GEOID to associate real estate data with spatial.
- [ ] Update the UI to display various real estate stats.

DEBUG
Something weird is happening on the merge. 
- One issue is that for the airport the values are all NULL. That makes sense
  because there are no homes for sale. However, the distance calculation is 
  messing up.
- I switched the merge type to 'left' because the airport isn't included otherwise.
- The left merge may be doing something unexpected with duplicate rows or something.
- When visualizing the merged data set, Minneapolis is rendered as if there are multiple layers.
  Look at the merge type options. Perhaps save the merged DF out as a file and look at 
  the shape of the data.
"""

"\nTODO\nMerge real estate data.\n- [X] Associate real estate ZIP codes with GEOIDs\n- [X] Import real estate data as a Pandas dataframe.\n- [ ] Find a way to merge on NAME + GEOID to associate real estate data with spatial.\n- [ ] Update the UI to display various real estate stats.\n\nDEBUG\nSomething weird is happening on the merge. \n- One issue is that for the airport the values are all NULL. That makes sense\n  because there are no homes for sale. However, the distance calculation is \n  messing up.\n- I switched the merge type to 'left' because the airport isn't included otherwise.\n- The left merge may be doing something unexpected with duplicate rows or something.\n- When visualizing the merged data set, Minneapolis is rendered as if there are multiple layers.\n  Look at the merge type options. Perhaps save the merged DF out as a file and look at \n  the shape of the data.\n"

In [190]:
METERS_TO_MILES = 1609
MAX_DISTANCE_TO_AP = 20

# Project the geospatial data to a projected CRS to enable distance calculations.
mn_counties_shapes = mn_counties_shapes.to_crs("EPSG:4087")

# Find the centroid of all the shapes.
mn_counties_shapes['centroid'] = mn_counties_shapes.geometry.centroid

# The international airport is in Fort Snelling.
airport = mn_counties_shapes.loc[mn_counties_shapes['NAME'] == 'Fort Snelling']
airport_point = airport.iat[0, 11] # Get the centroid of the airport.

# Find the centroid of the Oronoco city limits.
# Note: The city of Oronoco is in the county of Oronoco.
oronoco = mn_counties_shapes[(mn_counties_shapes['NAME'] == 'Oronoco') & (mn_counties_shapes['COUSUBFP'] == '48598') ]
oronoco_point = oronoco.iat[0, 11]

# Find the distanace of all the shapes to the airport.
# The distance calculation is done in meters which is then converted to miles.
# 1 meter = 0.000621371 miles
# Divide # meters by 1609 to get the # miles.
# Note: There is probably a way to format the numbers at render time rather than round at calculation time.
mn_counties_shapes['miles_to_ap'] = round(mn_counties_shapes['centroid'].distance(airport_point)/METERS_TO_MILES,1)
mn_counties_shapes['miles_to_oronoco'] = round(mn_counties_shapes['centroid'].distance(oronoco_point)/METERS_TO_MILES, 1)

# Project to the Mercator CRS for visualization.
mn_counties_shapes = mn_counties_shapes.to_crs("EPSG:3857")

# Find the additional areas we want to have on the map
additional_areas = mn_counties_shapes[(mn_counties_shapes['NAME'] == 'Oronoco') & (mn_counties_shapes['COUSUBFP'] == '48598') ]

# Find all the areas that are withing the maximum distance to the airport.
close_to_ap = mn_counties_shapes[mn_counties_shapes['miles_to_ap'] <= MAX_DISTANCE_TO_AP]

print (f'Filtering by Maximum Distance to Airport: {MAX_DISTANCE_TO_AP}')
print(f'Viewing {len(close_to_ap.index)} neighborhoods.')

# Create an interactive map using the Folium library.
# folium.Map
close_to_ap_map = close_to_ap.explore(
  column='miles_to_ap', 
  tooltip=['NAME','miles_to_ap', 'miles_to_oronoco'], 
  legend=True,
  name='neighborhoods',
  cmap='gist_earth' #YlOrRd, hot, copper
)
additional_areas.explore(
  m=close_to_ap_map, 
  column='miles_to_ap',
  tooltip=['NAME','miles_to_ap'],
  legend=False,
  color='blue',
  name='areas of interest'
)

folium.LayerControl().add_to(close_to_ap_map)  # use folium to add layer control
display(close_to_ap_map)
print(close_to_ap['NAME'])

Filtering by Maximum Distance to Airport: 20
Viewing 67 neighborhoods.


/Users/sholloway/Documents/Code/mn_neighborhoods/.venv/lib/python3.9/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam


117               Blaine
197     Spring Lake Park
330           North Oaks
389            Richfield
560              Hilltop
              ...       
2622              Empire
2623             Mendota
2651        Eden Prairie
2724       Medicine Lake
2752            Gem Lake
Name: NAME, Length: 67, dtype: object
